In [2]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import requests
import pandas as pd
import time
from datetime import datetime

# --- PARÁMETROS ESPECÍFICOS PARA TU TAREA ---
GENRES = ['funk']
YEARS = [2000, 2024] 

In [3]:
pip install python-dotenv

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
import os
from dotenv import load_dotenv

load_dotenv() # Carga las variables del archivo .env

SPOTIPY_CLIENT_ID = os.getenv('SPOTIPY_CLIENT_ID')
SPOTIPY_CLIENT_SECRET = os.getenv('SPOTIPY_CLIENT_SECRET')
LASTFM_API_KEY = os.getenv('LASTFM_API_KEY')

In [22]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

# 1. Años para iterar
YEARS = [2000, 2024]

# 2. Lista de géneros relacionados con funk (Python, no texto)
funk_related_genres = [
    # Funk clásico
    "funk",
    "classic funk",
    "deep funk",
    "funky",
    "funky breaks",
    "p-funk",
    "funk rock",
    "funk metal",
    "afro funk",
    "jazz funk",
    "disco funk",
    "boogie",
    "old school funk",

    # Soul / neo soul
    "soul",
    "classic soul",
    "neo soul",
    "contemporary r&b",
    "motown",
    "funk soul",
    
    # Funk house / dance
    "funk house",
    "disco house",
    "funky house",
    "deep funk house",
    "electro funk",
    "dance funk",
    "groove house"
]

# 3. Términos optimizados para búsqueda (puedes usar directamente la lista anterior)
query_terms = funk_related_genres

# 4. Inicializa el cliente de Spotify
SPOTIPY_CLIENT_ID = "TU_CLIENT_ID_AQUI"
SPOTIPY_CLIENT_SECRET = "TU_CLIENT_SECRET_AQUI"

try:
    sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(
        client_id=SPOTIPY_CLIENT_ID,
        client_secret=SPOTIPY_CLIENT_SECRET
    ))
    print("✅ Conexión a Spotify exitosa.")
except Exception as e:
    print(f"❌ Error de conexión a Spotify: {e}")
    sp = None


✅ Conexión a Spotify exitosa.


In [23]:
# NUEVA CELDA: Comprobación de autenticación
print(type(sp)) 

<class 'spotipy.client.Spotify'>


In [31]:
def extract_spotify_data_funk_ampliada(
    sp,
    years,
    query_terms,
    max_total_songs=500,
    max_songs_per_year=250
): 
    """
    Itera por AÑO y por cada TÉRMINO de género.
    - Limita el total global de canciones a max_total_songs.
    - Limita el máximo por año a max_songs_per_year.
    """
    spotify_data = []
    total_songs = 0  # contador global

    for year in years:
        year_range = f'{year}-{year}'
        songs_in_year = 0  # contador por año
        print(f"\n--- INICIANDO EXTRACCIÓN PARA EL AÑO {year} ---")

        for term in query_terms:
            # Si ya se alcanzó el límite global o el del año, paramos
            if total_songs >= max_total_songs:
                print("\n⚠️ Límite global de canciones alcanzado. Deteniendo extracción.")
                return pd.DataFrame(spotify_data)

            if songs_in_year >= max_songs_per_year:
                print(f"⚠️ Límite de canciones para el año {year} alcanzado ({songs_in_year}). Pasando al siguiente año.")
                break

            query = f'genre:"{term}" year:{year_range}'
            print(f"   -> Buscando con término: {term}")

            try:
                results = sp.search(q=query, type='track', limit=50)
                total_resultados_api = results['tracks']['total'] if results and 'tracks' in results else 0
                print(f"   -> Spotify reporta {total_resultados_api} resultados para '{term}'.")

                while results and total_songs < max_total_songs and songs_in_year < max_songs_per_year:
                    for track in results['tracks']['items']:
                        track_id = track['id']

                        # Evitar duplicados
                        if any(d.get('ID_Spotify') == track_id for d in spotify_data):
                            continue

                        if total_songs >= max_total_songs or songs_in_year >= max_songs_per_year:
                            break

                        data_row = {
                            'Artista': track['artists'][0]['name'],
                            'Género musical': f'funk (Origen: {term})',
                            'Tipo': 'Canción',
                            'Nombre': track['name'],
                            'Año de lanzamiento': year,
                            'ID_Spotify': track_id,
                            'ID_Album': track['album']['id']
                        }
                        spotify_data.append(data_row)
                        total_songs += 1
                        songs_in_year += 1

                    # Paginación
                    if results['tracks']['next'] and total_songs < max_total_songs and songs_in_year < max_songs_per_year:
                        results = sp.next(results['tracks'])
                    else:
                        results = None

                    time.sleep(0.5)

                print(f"      -> Acumulado año {year}: {songs_in_year} | Total global: {total_songs}")

            except Exception as e:
                print(f"   -> Error al buscar con término '{term}': {e}")
                time.sleep(2)

    return pd.DataFrame(spotify_data)


# --- LLAMADA ---
if sp:
    df_spotify_raw = extract_spotify_data_funk_ampliada(
        sp,
        YEARS,
        query_terms,
        max_total_songs=500,    # máximo total
        max_songs_per_year=250  # máximo por año
    )
    print("\n--- RESULTADO FINAL DE FUNK (ESTRATEGIA ROBUSTA) ---")
    print(f"Total de registros extraídos: {len(df_spotify_raw)}")
    print(df_spotify_raw.head(10))



--- INICIANDO EXTRACCIÓN PARA EL AÑO 2000 ---
   -> Buscando con término: funk
   -> Spotify reporta 16 resultados para 'funk'.
      -> Acumulado año 2000: 16 | Total global: 16
   -> Buscando con término: classic funk
   -> Spotify reporta 1 resultados para 'classic funk'.
      -> Acumulado año 2000: 16 | Total global: 16
   -> Buscando con término: deep funk
   -> Spotify reporta 0 resultados para 'deep funk'.
      -> Acumulado año 2000: 16 | Total global: 16
   -> Buscando con término: funky
   -> Spotify reporta 0 resultados para 'funky'.
      -> Acumulado año 2000: 16 | Total global: 16
   -> Buscando con término: funky breaks
   -> Spotify reporta 0 resultados para 'funky breaks'.
      -> Acumulado año 2000: 16 | Total global: 16
   -> Buscando con término: p-funk
   -> Spotify reporta 1 resultados para 'p-funk'.
      -> Acumulado año 2000: 16 | Total global: 16
   -> Buscando con término: funk rock
   -> Spotify reporta 0 resultados para 'funk rock'.
      -> Acumulado añ

In [30]:
# --- LLAMADA AL CÓDIGO ---

if sp:
    # Pasamos la lista de términos como argumento
    df_spotify_raw = extract_spotify_data_funk_ampliada(sp, YEARS, query_terms, max_total_songs=500)
    print("\n--- RESULTADO FINAL DE FUNK (ESTRATEGIA ROBUSTA) ---")
    print(f"Total de registros extraídos: {len(df_spotify_raw)} (Nota: este número ya elimina duplicados entre términos)")
    print(df_spotify_raw.head(10))


--- INICIANDO EXTRACCIÓN PARA EL AÑO 2000 ---
   -> Buscando con término: funk
   -> Spotify reporta 47 resultados para 'funk'.
      -> Añadidas 47 canciones de 'funk'.
   -> Buscando con término: classic funk
   -> Spotify reporta 100 resultados para 'classic funk'.
      -> Añadidas 95 canciones de 'classic funk'.
   -> Buscando con término: deep funk
   -> Spotify reporta 17 resultados para 'deep funk'.
      -> Añadidas 17 canciones de 'deep funk'.
   -> Buscando con término: funky
   -> Spotify reporta 49 resultados para 'funky'.
      -> Añadidas 26 canciones de 'funky'.
   -> Buscando con término: funky breaks
   -> Spotify reporta 4 resultados para 'funky breaks'.
      -> Añadidas 3 canciones de 'funky breaks'.
   -> Buscando con término: p-funk
   -> Spotify reporta 100 resultados para 'p-funk'.
      -> Añadidas 68 canciones de 'p-funk'.
   -> Buscando con término: funk rock
   -> Spotify reporta 100 resultados para 'funk rock'.
      -> Añadidas 63 canciones de 'funk rock

In [27]:
# --- NUEVA FUNCIÓN: EXTRACCIÓN DE DETALLES DEL ÁLBUM ---

def extract_album_details(sp, album_ids):
    """
    Extrae los metadatos completos (nombre, año, artista) de los álbumes.
    """
    album_details = []
    
    # Usamos un set para IDs de álbumes ya procesados
    unique_album_ids = set(album_ids)
    
    print(f"\n--- INICIANDO EXTRACCIÓN DE METADATOS DE {len(unique_album_ids)} ÁLBUMES ---")
    
    for album_id in unique_album_ids:
        try:
            # Endpoint de Spotify para obtener detalles de un álbum específico
            album_info = sp.album(album_id) 
            
            data_row = {
                'ID_Album': album_id,
                'Nombre_Album': album_info['name'],
                'Año_Lanzamiento_Album': album_info['release_date'][:4], # Tomamos solo el año
                'Artista_Principal_Album': album_info['artists'][0]['name'],
                'Tipo_Lanzamiento': album_info['album_type'] # (album, single, compilation)
            }
            album_details.append(data_row)
            time.sleep(0.2) # Pausa ligera
            
        except Exception as e:
            # Imprime un mensaje si un álbum falla (ej. si fue eliminado de Spotify)
            print(f"   -> Error al extraer detalles del álbum {album_id}: {e}")
            pass
            
    print(f"✅ Extracción de {len(album_details)} álbumes completada.")
    return pd.DataFrame(album_details)